In [1]:
import numpy as np
import os

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"]="2"

In [3]:
import PIL
import tensorflow as tf
import random
from tensorflow.python.framework.ops import disable_eager_execution
from tensorflow.python.framework.ops import enable_eager_execution
#disable_eager_execution()
enable_eager_execution()

In [4]:
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.applications.resnet50 import ResNet50
import tensorflow_datasets as tfds

In [5]:
from __future__ import print_function
import argparse
from tensorflow.keras.layers import Input
import scipy.misc

from configs import bcolors
from utils import *
import tensorflow_model_optimization as tfmot

In [6]:
def preprocess_image(features):
    """Preprocesses the given image.

      Args:
        image: `Tensor` representing an image of arbitrary size.

      Returns:
        A preprocessed image `Tensor` of range [0, 1].
  """
    image = features["image"]
    image = tf.image.resize(image,[224,224])
    image = tf.image.convert_image_dtype(image, dtype=tf.float32)
    image = tf.keras.applications.resnet.preprocess_input(image)
    #image = center_crop(image, 224, 224, crop_proportion=0.875)   # Standard for ImageNet.
    #image = tf.reshape(image, [224, 224, 3])
    #image = tf.clip_by_value(image, 0., 1.)
    
    features["image"] = image
    return features

# Load Data

In [7]:
BATCH_SIZE = 50
tfds_dataset2, tfds_info  = tfds.load(name='imagenet2012_subset', split='validation[-20%:]', with_info=True,
                                     data_dir='/local/rcs/wei/image_net/')
#tf.compat.v1.data.make_one_shot_iterator(tfds_dataset1).get_next()

In [ ]:
figs = tfds.show_examples(tfds_dataset2, tfds_info)

In [ ]:
val_ds = tfds_dataset2.map(preprocess_image).batch(BATCH_SIZE).prefetch(1)

# Load Model

In [7]:
# input image dimensions
img_rows, img_cols = 224 ,224
input_shape = (img_rows, img_cols, 3)
model_ = ResNet50(input_shape=input_shape)

In [8]:
q_model = tfmot.quantization.keras.quantize_model(model_)

In [9]:
model = ResNet50(input_tensor = q_model.get_layer('input_1').input)

In [10]:
model.load_weights("./original_model.h5")
q_model.load_weights("./q_model.h5")

# Display Image

In [14]:
def extract():
    for i, features in enumerate(val_ds):
        image = np.expand_dims(features["image"].numpy()[0], axis=0)
        label = features["label"].numpy()[0]
        logits_1 = model(image).numpy()
        logits_2 = q_model(image).numpy()
        predict_1 = tf.argmax(logits_1, axis=-1).numpy()
        predict_2 = tf.argmax(logits_2, axis=-1).numpy()
        break
    return image,label,logits_1,logits_2,predict_1,predict_2

In [15]:
image,label,logits_1,logits_2,predict_1,predict_2 = extract()

RuntimeError: __iter__() is only supported inside of tf.function or when eager execution is enabled.

In [16]:
pil_img = tf.keras.preprocessing.image.array_to_img(image[0,:,:,:])
from IPython.display import Image
display(pil_img)

TypeError: 'module' object is not subscriptable

In [ ]:
tf.keras.applications.resnet.decode_predictions(logits_1, top=5)[0]

In [ ]:
tf.keras.applications.resnet.decode_predictions(logits_2, top=5)[0]

In [ ]:
label

In [ ]:
np.save("./trailer_truck.npy",image)

# Generate adverserial example

In [11]:
image = np.load("./trailer_truck.npy")

In [12]:
weight_diff = 10
orig_label = 867
grad_iterations = 20
transformation = 'light'
step = 10

In [16]:
orig_img = image.copy()
for iters in range(0,grad_iterations):
    input_image = tf.convert_to_tensor(orig_img)

    with tf.GradientTape() as g:
        g.watch(input_image)
        loss1 = K.mean(model(input_image)[..., orig_label])
        loss2 = K.mean(q_model(input_image)[..., orig_label])
        final_loss = K.mean(loss1 - weight_diff*loss2)
        #A = argmin L(F(image + A) , y) - c* L(f(image + A) , y_1) 

    grads = normalize(g.gradient(final_loss, input_image)[0])

    #grads_value = iterate([image])[0]
    if transformation == 'light':
        grads = constraint_light(grads)  # constraint the gradients value
    elif transformation == 'occl':
        grads = constraint_occl(grads, start_point,occlusion_size)  # constraint the gradients value
    elif transformation == 'blackout':
        grads = constraint_black(grads)  # constraint the gradients value

    image += grads * step    
    pred1, pred2= model.predict(image), q_model.predict(image)
    label1, label2 = np.argmax(pred1[0]), np.argmax(pred2[0])
    if not label1 == label2:
        print("different")
        print(label1)
        print(label2)
        print(tf.keras.applications.resnet.decode_predictions(pred1, top=5)[0])
        print(tf.keras.applications.resnet.decode_predictions(pred2, top=5)[0])
        gen_img_deprocessed = deprocess_image(image)
        orig_img_deprocessed = deprocess_image(orig_img)
        break

different
565
867
[('n03393912', 'freight_car', 0.47857615), ('n04467665', 'trailer_truck', 0.4567812), ('n03417042', 'garbage_truck', 0.017796256), ('n04310018', 'steam_locomotive', 0.014601685), ('n03796401', 'moving_van', 0.013246967)]
[('n04467665', 'trailer_truck', 0.640401), ('n03393912', 'freight_car', 0.200086), ('n03417042', 'garbage_truck', 0.07589043), ('n03796401', 'moving_van', 0.038500518), ('n04310018', 'steam_locomotive', 0.013253607)]


In [23]:
pred1, pred2= model.predict(image), q_model.predict(image)

In [24]:
tf.keras.applications.resnet.decode_predictions(pred1, top=5)[0]

[('n03393912', 'freight_car', 0.546804),
 ('n04252225', 'snowplow', 0.23667075),
 ('n04467665', 'trailer_truck', 0.06796761),
 ('n03417042', 'garbage_truck', 0.051419698),
 ('n04310018', 'steam_locomotive', 0.017259402)]

In [25]:
tf.keras.applications.resnet.decode_predictions(pred2, top=5)[0]

[('n04252225', 'snowplow', 0.6547978),
 ('n04310018', 'steam_locomotive', 0.07759655),
 ('n04467665', 'trailer_truck', 0.06391995),
 ('n03417042', 'garbage_truck', 0.05801411),
 ('n03393912', 'freight_car', 0.043373536)]

In [ ]:
tf.keras.preprocessing.image.array_to_img(gen_image[0,:,:,:])

In [ ]:
tf.keras.preprocessing.image.array_to_img(gen_img_deprocessed)

In [ ]:
tf.keras.preprocessing.image.array_to_img(orig_img_deprocessed)